# NLP to SQL

In this project, we'll develop a start-up that can take a non-technical manager's Natural Language question, such as "What counties had the top sales?" and convert that into a SQL query. We will then use that SQL to query the data (which in this example will come from a .csv file we read in with Pandas and set-up a temporary DB in RAM) and then report back the results!

The dataset we will use is a Kaggle Dataset

https://www.kaggle.com/datasets/kyanyoga/sample-sales-data

## Library Imports

In [23]:
import os
import pandas as pd
import openai as openai

In [24]:
# Lets read the dataset
df = pd.read_csv("../resources/sales_data_sample.csv")

# check the data load
df.head(5)

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,...,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE
0,10107,30,95.70,2,2871.00,2/24/2003 0:00,Shipped,1,2,2003,...,897 Long Airport Avenue,NaN,NYC,NY,10022,USA,NaN,Yu,Kwai,Small
1,10121,34,81.35,5,2765.90,5/7/2003 0:00,Shipped,2,5,2003,...,59 rue de l'Abbaye,NaN,Reims,NaN,51100,France,EMEA,Henriot,Paul,Small
2,10134,41,94.74,2,3884.34,7/1/2003 0:00,Shipped,3,7,2003,...,27 rue du Colonel Pierre Avia,NaN,Paris,NaN,75508,France,EMEA,Da Cunha,Daniel,Medium
3,10145,45,83.26,6,3746.70,8/25/2003 0:00,Shipped,3,8,2003,...,78934 Hillside Dr.,NaN,Pasadena,CA,90003,USA,NaN,Young,Julie,Medium
4,10159,49,100.00,14,5205.27,10/10/2003 0:00,Shipped,4,10,2003,...,7734 Strong St.,NaN,San Francisco,CA,NaN,USA,NaN,Brown,Julie,Medium


## Querying the Data 

We can query the data via pandas syntax

In [25]:
# sum of sales by Quarter
df.groupby('QTR_ID').sum()['SALES']

QTR_ID
1    2350817.73
2    2048120.30
3    1758910.81
4    3874780.01
Name: SALES, dtype: float64

## SQL Database Setup

In [26]:
from sqlalchemy import create_engine
from sqlalchemy import text

In [27]:
temp_db = create_engine('sqlite:///:memory:', echo=True)

In [28]:
# Now we are pushing our entire dataframe to a table called Sales

data = df.to_sql(name='Sales', con=temp_db)

2023-10-01 18:53:10,970 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-01 18:53:10,979 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Sales")
2023-10-01 18:53:10,980 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-10-01 18:53:10,982 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Sales")
2023-10-01 18:53:10,983 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-10-01 18:53:10,987 INFO sqlalchemy.engine.Engine 
CREATE TABLE "Sales" (
	"index" BIGINT, 
	"ORDERNUMBER" BIGINT, 
	"QUANTITYORDERED" BIGINT, 
	"PRICEEACH" FLOAT, 
	"ORDERLINENUMBER" BIGINT, 
	"SALES" FLOAT, 
	"ORDERDATE" TEXT, 
	"STATUS" TEXT, 
	"QTR_ID" BIGINT, 
	"MONTH_ID" BIGINT, 
	"YEAR_ID" BIGINT, 
	"PRODUCTLINE" TEXT, 
	"MSRP" BIGINT, 
	"PRODUCTCODE" TEXT, 
	"CUSTOMERNAME" TEXT, 
	"PHONE" TEXT, 
	"ADDRESSLINE1" TEXT, 
	"ADDRESSLINE2" TEXT, 
	"CITY" TEXT, 
	"STATE" TEXT, 
	"POSTALCODE" TEXT, 
	"COUNTRY" TEXT, 
	"TERRITORY" TEXT, 
	"CONTACTLASTNAME" TEXT, 
	"CONTACTFIRSTNAME" TEXT, 
	"DEALSIZE" TEXT
)


20

### Connecting to SQL Database

In [29]:
# Using SQL Alchemy  we can establish a connection  to this temporary database and query it for the results

with temp_db.connect() as conn:
    result = conn.execute(text('SELECT ORDERNUMBER, SALES FROM Sales ORDER BY SALES DESC LIMIT 3'))

result.all()

2023-10-01 18:53:14,367 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-01 18:53:14,368 INFO sqlalchemy.engine.Engine SELECT ORDERNUMBER, SALES FROM Sales ORDER BY SALES DESC LIMIT 3
2023-10-01 18:53:14,369 INFO sqlalchemy.engine.Engine [generated in 0.00246s] ()
2023-10-01 18:53:14,371 INFO sqlalchemy.engine.Engine ROLLBACK


[(10407, 14082.8), (10322, 12536.5), (10424, 12001.0)]

## OPENAI API

### Get The OpenAI API Key

In [30]:
openai.api_key = os.environ["OPENAI_API_KEY"]

### Inform GPT about the SQL Table Structure

We need to tell GPT what the table structure looks like before it can understand the schema enough to create a SQL query. 

- Automatically creating table structure  definition from Data frame  for OpenAI API
- Grabbing user NLP input
- Combining prompts for an OpenAI API call

an OPENAI GPT Table description looks like this ...

In [31]:
### sqllite <here we have to mention database type, SQLServer, Postgres etc> SQL Tables, with their properties:
#
# Employee (id, name, department_id)
# Department (id, name, address)
# Salary_Payments (id, employee_id, amount, date)
#
### A Query to list the names of the departments which employed more than 10 employees  in the last 3 months
# SELECT (at the end we tell it to "SELECT", Thus GPT must finish with the rest of the most reasonable SQL query)

In [32]:
# Instead of giving prompts like above for every table, we create a function which will automatically create the table definition prompt

def create_table_defintion_prompt(df):
    """
    This function returns a prompt that informs GPT that we want to work with SQL Tables
    """
    prompt = '''### sqllite SQL table with its properties:
#    
# Sales({})
#
'''.format(",".join(str(columnnames) for columnnames in df.columns)) # format call injects all the columns
    return prompt

In [33]:
# Lets test the above function
print(create_table_defintion_prompt(df))

### sqllite SQL table with its properties:
#    
# Sales(ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,MSRP,PRODUCTCODE,CUSTOMERNAME,PHONE,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE)
#



### Get Natural Language Request
Now let's create a function that grabs the natural language information request:

In [34]:
# Function to grab the user query
def prompt_input():
    nlp_text = input("Enter information you want to obtain: ")
    return nlp_text

In [35]:
# Lets test the function ... 
nlp_text = prompt_input()
print(nlp_text)

sum of SALES per POSTALCODE


In [36]:
# Now we will combine the results in one function
def combine_prompts(df, query_prompt):
    definition = create_table_defintion_prompt(df)
    query_init_string = f"### A Query to answer: {query_prompt}\n # SELECT"
    return (definition + query_init_string)

In [38]:
# Lets test the combined function
print(combine_prompts(df, "Sum of SALES per POSTALCODE"))

### sqllite SQL table with its properties:
#    
# Sales(ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,MSRP,PRODUCTCODE,CUSTOMERNAME,PHONE,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE)
#
### A Query to answer: Sum of SALES per POSTALCODE
 # SELECT


### OpenAPI API Call - Completion API Calls

Currently there are two main types of Completion API Calls

- Text Completion
    - Optimized for completing natural language
- Code Completion
    - Optimized for completing with actual executable code

The following models are in GPT 3.5
- code-davinci-002
- text-davinci-002
- text-davinci-003

#### OpenAPI API Completion Call Parameters

- **Model**                     - Name of the model to be used
- **Prompt**                    - Most important parameter. Basically dictates what the model is supposed to return.
- **Temperature**               - Higher the value means, model is going to take more risk. The model will include lower probabilities (range between 0 and 1) - ex 0.9 for creative answer, 0.1 for obvious simple answers
- **Max Tokens**                - Max tokens to generate in the completion. Most models have context length 2048, however newest models support 4096
- **Top P**                     - An Alternative to sampling with temperature, called nucleus sampling. will consider top_p probability mass.So 0.1 means consider to 10% probability mass. Alter Top P or Temperature but not both.
- **N**                         - How many completions to generate for each prompt. Pretty much running the same prompt multiple times. We essentially almost always set this to 1.
- **Frequency Penalty**         - Between -2.0 to 2.0. Essentially to avoid GPT models give the same response again and again from prompts are getting fine tuned. Positive values penalize the new tokens based on their existing frequency, decreasing the model's liklihood to repeat the same line verbatim.
- **Presence Penalty**          - Between -2.0 to 2.0. Essentially to avoid GPT models give the same response again and again from prompts are getting fine tuned. Positive values penalize the new tokens based on whether they appear in the text so far, increasing the model's liklihood to talk about new  topics.

The presence penalty is a one off additive contribution that applies to all tokens that have been sampled at least once and the frequency penalty is a contribution that is proportional to how often a particular token hasalready been sampled.
Reasonable values for the penalty coefficients are typically around 0.1 to 1 if the aim is just reduce repetitive samples somewhat. If the idea is to be really restrictive, then we can go beyond 1 to 2. But this might degrade the quality of the samples.
Negative values can be used to increase the likelihood of repetition.

In [48]:
# We will first use the combine_prompts function to quickly check our combined prompt to the GPT
print(combine_prompts(df, nlp_text))

### sqllite SQL table with its properties:
#    
# Sales(ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,MSRP,PRODUCTCODE,CUSTOMERNAME,PHONE,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE)
#
### A Query to answer: sum of SALES per POSTALCODE
 # SELECT


In [43]:
response = openai.Completion.create(
    model = 'text-davinci-003',
    prompt = combine_prompts(df, nlp_text),
    temperature = 0,
    max_tokens = 150,
    top_p = 0.1,
    frequency_penalty = 0.0,
    presence_penalty = 0.0,
    stop = ["#", ";"]
)

In [44]:
# Lets check the response
response

<OpenAIObject text_completion id=cmpl-84qmrACCjyq3INdju7N9gNB7xK228 at 0x7f8a96943ce0> JSON: {
  "warning": "This model version is deprecated. Migrate before January 4, 2024 to avoid disruption of service. Learn more https://platform.openai.com/docs/deprecations",
  "id": "cmpl-84qmrACCjyq3INdju7N9gNB7xK228",
  "object": "text_completion",
  "created": 1696166917,
  "model": "text-davinci-003",
  "choices": [
    {
      "text": " POSTALCODE, SUM(SALES) FROM Sales GROUP BY POSTALCODE",
      "index": 0,
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 158,
    "completion_tokens": 19,
    "total_tokens": 177
  }
}

In [51]:
response["choices"][0]["text"]

' POSTALCODE, SUM(SALES) FROM Sales GROUP BY POSTALCODE'

In [52]:
# Now we will build a function to parse the section of the response we want
def handle_response(response):
    query = response["choices"][0]["text"]
    if query.startswith(" "):
        query = "SELECT" + query
    return query

In [54]:
# Now we will verify our section of the complete response

query = handle_response(response)
query

'SELECT POSTALCODE, SUM(SALES) FROM Sales GROUP BY POSTALCODE'

In [55]:
# Now that we have got the query, we will pass that to our database
with temp_db.connect() as conn:
    result = conn.execute(text(handle_response(response)))

# see the result
result.all()

2023-10-01 19:10:54,865 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-01 19:10:54,866 INFO sqlalchemy.engine.Engine SELECT POSTALCODE, SUM(SALES) FROM Sales GROUP BY POSTALCODE
2023-10-01 19:10:54,867 INFO sqlalchemy.engine.Engine [generated in 0.00210s] ()
2023-10-01 19:10:54,870 INFO sqlalchemy.engine.Engine ROLLBACK


[(None, 272407.14),
 ('10022', 560787.7699999998),
 ('10100', 94117.26000000002),
 ('106-0032', 120562.73999999996),
 ('1203', 117713.55999999998),
 ('1227 MM', 94015.73),
 ('13008', 74936.14),
 ('1734', 145041.6),
 ('2', 57756.43),
 ('2060', 153996.13000000003),
 ('2067', 151570.98000000004),
 ('21240', 111250.37999999996),
 ('24067', 85555.98999999998),
 ('24100', 137955.72000000003),
 ('28023', 170257.33000000005),
 ('28034', 912294.1100000002),
 ('3004', 200995.40999999997),
 ('31000', 70488.44),
 ('3150', 64591.46000000001),
 ('4101', 59469.11999999999),
 ('4110', 116599.19),
 ('41101', 54723.62),
 ('42100', 142601.33000000002),
 ('44000', 204304.86),
 ('5020', 149798.63),
 ('50553', 207874.86),
 ('50739', 100306.58),
 ('51003', 154069.65999999997),
 ('51100', 135042.94),
 ('51247', 139243.99999999994),
 ('530-0003', 67605.07),
 ('58339', 165255.20000000004),
 ('59000', 69052.41),
 ('60528', 85171.58999999998),
 ('62005', 131685.30000000002),
 ('67000', 80438.48),
 ('69004', 14287